In [1]:
import os
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.core_plugins import SessionsPythonTool
from azure.identity import EnvironmentCredential, get_bearer_token_provider
from dotenv import load_dotenv

load_dotenv()
pool_management_endpoint = os.getenv("POOL_MANAGEMENT_ENDPOINT")
chat_endpoint=os.getenv("OPENAI_ENDPOINT")
model_deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME")

In [2]:
async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    for item in message.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"# Function Result:> {item.result}")
        elif isinstance(item, FunctionCallContent):
            print(f"# Function Call:> {item.name} with arguments: {item.arguments}")
        else:
            print(f"# {message.name}: {message} ")

### Azure Dyamic Session Example

Authentication is also handled with service principals - just need to ensure the SPN has a role assignment to the resource with the `Azure ContainerApps Session Executor` role.

The example below configures a `ChatCompletionAgent` backed with a `SessionsPythonTool` to execute python in a sandboxed environment.  
There are a few things we can configure with the pool:
- Language type: Python
- Maximum concurrent sessions: 5
- Session cooldown period: 300 seconds
- Network egress: disabled


In [5]:

# 1. Create the python code interpreter tool using the SessionsPythonTool
python_code_interpreter = SessionsPythonTool(pool_management_endpoint=pool_management_endpoint)

# 2. Create the agent
agent = ChatCompletionAgent(
    service=AzureChatCompletion(
        endpoint=chat_endpoint,
        ad_token_provider=get_bearer_token_provider(EnvironmentCredential(), "https://cognitiveservices.azure.com/.default"),
        deployment_name=model_deployment_name
    ),
    name="MathTutor",
    instructions="You are helping a 10 year old boy with his math homework.  ALWAYS use the python code interpreter to solve the problem.",
    plugins=[python_code_interpreter]
)

# 3. Upload a CSV file to the session
# file_metadata = await python_code_interpreter.upload_file(local_file_path="./test.txt")

TASK = "What's 2x + 1 = 10?"
print(f"# User: '{TASK}'")

async for response in agent.invoke(
    messages=TASK,
    on_intermediate_message=handle_intermediate_steps,
):
    print(f"# {response.name}: {response} ")

# User: 'What's 2x + 1 = 10?'
# Function Call:> SessionsPythonTool-execute_code with arguments: {"code":"from sympy import symbols, Eq, solve\nx = symbols('x')\nequation = Eq(2*x + 1, 10)\nsolution = solve(equation, x)\nsolution"}
# Function Result:> Status:
Success
Result:
[9/2]
Stdout:

Stderr:

# MathTutor: The solution to the equation \( 2x + 1 = 10 \) is \( x = \frac{9}{2} \), or in decimal form, \( x = 4.5 \). 
